In [3]:
# Here we set up Tensorflow for the Colab notebook

%tensorflow_version 1.x
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

Using TensorFlow backend.


In [4]:
# Next we take a CSV file and turn it into a text file
global inputDir, outputDir, trainingFilePath, allTitles

import csv
from random import shuffle

# Where you are storing your training files in your drive
inputDir = '/content/drive/My Drive/Colab Notebooks/training/color_names/'
outputDir = '/content/drive/My Drive/Colab Notebooks/output/color_names/'

# The CSV file you want to parse
csvFileName = 'color-names.csv'

# Turn it into a .txt for textgenrnn
trainingFileName = 'training-names.txt'

csvFilePath = os.path.join(inputDir, csvFileName)
trainingFilePath = os.path.join(inputDir, trainingFileName)

with open(csvFilePath) as csvFile:
  # setup a csv reader
  csvReader = csv.reader(csvFile)
  # skip first line (header)
  next(csvReader)
  # gather all the names
  colors = [ color for color in csvReader ]
  names = [ color[1].strip() + ' ' + color[0].strip() for color in colors ]
  # shuffle the data
  shuffle(names)
  print('Got %d titles' % len(names))

  # Get all titles as lower-case
  # We will use this later to see if a generated name
  # already exists in the data
  allTitles = [ color[0].strip().lower() for color in colors ]

  # write file
  with open(trainingFilePath, 'w') as out:
    # first line should be a header
    text = 'title\n'

    # then all the data split by lines
    text += '\n'.join(names)

    # finally write it out
    out.write(text)
    print('Writing %s' % trainingFilePath)


Got 18329 titles
Writing /content/drive/My Drive/Colab Notebooks/training/color_names/training-names.txt


In [7]:
global textgen

# get an output path for trained model data
weightsFilePath = os.path.join(outputDir, 'textgenrnn_weights_saved.hdf5')

# make directory if it doesn't exist
os.makedirs(outputDir, exist_ok=True)

print('Output Data %s' % weightsFilePath)

# Whether to train new weights or load previous weights
train_model = False
textgen = textgenrnn()

if train_model:
  print('Training...')
  model_config = {
      'num_epochs': 6,
      'batch_size': 1024
  }
  textgen.train_from_file(trainingFilePath, gen_epochs=0, progress=False, **model_config)
  textgen.save(weightsFilePath)
  print('Training complete!')
else:
  textgen.load(weightsFilePath)
  print('Loaded previous session!')

Output Data /content/drive/My Drive/Colab Notebooks/output/color_names/textgenrnn_weights_saved.hdf5
Loaded previous session!


In [13]:
def gen_samples (model, n=5, temperatures=[0.5, 0.8, 1.0]):
  for temperature in temperatures:
    print("----- TEMP %s" % temperature)
    list = model.generate(n=n, temperature=temperature, return_as_list=True)
    for item in list:
      postfix = ''
      # get everything after the color code
      name = ' '.join(item.split(' ')[1:])
      if name.lower() in allTitles:
        postfix = ' [Exists in Data]'
      print(item + postfix)
    print()

gen_samples(textgen)

----- TEMP 0.5
#705c71 Peach Clay
#89cecb Strawberry Passion
#b9968c Steel Blue [Exists in Data]
#8f4c5c Cute Lilac
#9b9b9c Bright Cream

----- TEMP 0.8
#fe6caf Pool North
#678631 Eclipse [Exists in Data]
#667778 Lemon Bay
#ecad97 Grapingano
#e8b55a Matmous Shady

----- TEMP 1.0
#54785f Radian Green
#e9cf15 Asalite Green
#e5acf2 Wisteria Glass
#b8b645 Pink Leaf
#9d7061 Deep Moss

